In [1]:
import json
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# Fonction pour lire les données de transcription
def read_transcription(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Fonction pour lire les données du graphe de discours
def read_discourse_graph(file_path):
    with open(file_path, 'r') as file:
        data = [line.strip().split() for line in file]

    # Convertir les valeurs non numériques en indices numériques
    data = [(int(start), relation, int(end)) if start.isdigit() and end.isdigit() else (start, relation, end) for start, relation, end in data]

    return data

def create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict):
    rows = []

      # Iterate through all edges in the discourse graph
    for edge in discourse_graph:
        index_start, relation_type, index_end = edge

        # Retrieve speaker information
        speaker = transcription[index_start]['speaker']

        # Convert relation type to integer using the dictionary
        speaker_id = speaker_dict.get(speaker, -1)

        # Retrieve the sentence
        text = transcription[index_start]['text']

        # Convert relation type to integer using the dictionary
        relation_type_id = relation_dict.get(relation_type, -1)

        # Add a row to the DataFrame
        rows.append({
            'dialogue_id': dialogue_id,
            'index_start': index_start,
            'text': text,
            'index_end': index_end,
            'speaker_type': speaker_id,
            'speaker_text': speaker,
            'relation_type': relation_type_id,
            'relation_text': relation_type
        })

    # Create the DataFrame
    df = pd.DataFrame(rows)

    return df


# Fonction pour créer le dictionnaire de conversion des relations
def create_relation_dict(discourse_graph):
    relation_set = set()

    # Collecter toutes les relations uniques
    for edge in discourse_graph:
        relation_set.add(edge[1])

    # Créer un dictionnaire de conversion
    relation_dict = {relation: idx for idx, relation in enumerate(relation_set)}

    return relation_dict

# Fonction pour créer le dictionnaire de conversion des speakers
def create_speaker_dict(transcription):
    speaker_set = set()

    # Collecter tous les locuteurs uniques
    for utterance in transcription:
        speaker_set.add(utterance['speaker'])

    # Créer un dictionnaire de conversion
    speaker_dict = {speaker: idx for idx, speaker in enumerate(speaker_set)}

    return speaker_dict

def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

# Function to get labels for a dialogue
def get_label(dialogue_id, index,labels_data):
    return labels_data.get(dialogue_id, [])[index]


In [3]:
# Remplacez 'votre_chemin' par le chemin correct
chemin_dossier = Path("data/training")

# Remplacez 'vos_dialogue_ids' par votre liste réelle d'identifiants de dialogue
dialogue_ids = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
dialogue_ids = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in dialogue_ids])
dialogue_ids.remove('IS1002a')
dialogue_ids.remove('IS1005d')
dialogue_ids.remove('TS3012c')

# Liste pour stocker les DataFrames de chaque dialogue
dfs = []

In [4]:
# Liste pour stocker les DataFrames de chaque dialogue
dfs = []

# Parcourir chaque dialogue
for dialogue_id in dialogue_ids:
    # Lire les données de transcription et de graphe de discours
    transcription = read_transcription(chemin_dossier / f'{dialogue_id}.json')
    discourse_graph = read_discourse_graph(chemin_dossier / f'{dialogue_id}.txt')
    
    # Créer le dictionnaire de conversion des relations
    relation_dict = create_relation_dict(discourse_graph)
    speaker_dict = create_speaker_dict(transcription)

    # Créer le DataFrame pour le dialogue actuel
    df = create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict)
    
    # Ajouter le DataFrame à la liste
    dfs.append(df)

    # Ajouter la dernière phrase avec NaN pour index_end et 'relation'
    last_utterance = transcription[-1]
    last_speaker = last_utterance['speaker']
    last_text = last_utterance['text']
    last_row = {
        'dialogue_id': dialogue_id,
        'index_start': len(transcription) - 1,
        'text': last_text,
        'index_end': 0,
        'speaker_type': speaker_dict.get(last_speaker, -1),
        'speaker_text': last_speaker,
        'relation_type': 0,
        'relation_text': np.nan
    }
    dfs.append(pd.DataFrame([last_row]))

In [5]:
# Concaténer tous les DataFrames en un seul
df = pd.concat(dfs, ignore_index=True)

with open("data/training_labels.json", 'r') as file:
    labels_data = json.load(file)

df['label'] = df.apply(lambda row: get_label(row['dialogue_id'], row['index_start'], labels_data), axis=1)


# Afficher le DataFrame final
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72623 entries, 0 to 72622
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dialogue_id    72623 non-null  object 
 1   index_start    72623 non-null  int64  
 2   text           72623 non-null  object 
 3   index_end      72526 non-null  float64
 4   speaker_type   72623 non-null  int64  
 5   speaker_text   72623 non-null  object 
 6   relation_type  72526 non-null  float64
 7   relation_text  72526 non-null  object 
 8   label          72623 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 5.0+ MB


,dialogue_id,index_start,text,index_end,speaker_type,speaker_text,relation_type,relation_text,label
0,ES2002a,0,Okay,1.0,3,PM,1.0,Continuation,0
1,ES2002a,1,Right,2.0,3,PM,1.0,Continuation,0
2,ES2002a,2,<vocalsound> Um well this is the kick-off meet...,3.0,3,PM,3.0,Explanation,1
3,ES2002a,3,Um <vocalsound> and um,4.0,3,PM,8.0,Elaboration,0
4,ES2002a,4,this is just what we're gonna be doing over th...,5.0,3,PM,1.0,Continuation,0


In [6]:
print(df[df['dialogue_id'] == 'ES2002a'])


    dialogue_id  index_start  \
0       ES2002a            0   
1       ES2002a            1   
2       ES2002a            2   
3       ES2002a            3   
4       ES2002a            4   
..          ...          ...   
391     ES2002a          391   
392     ES2002a          391   
393     ES2002a          393   
394     ES2002a          394   
395     ES2002a          395   

                                                  text  index_end  \
0                                                 Okay        1.0   
1                                                Right        2.0   
2    <vocalsound> Um well this is the kick-off meet...        3.0   
3                               Um <vocalsound> and um        4.0   
4    this is just what we're gonna be doing over th...        5.0   
..                                                 ...        ...   
391  Right , okay , we'll that's that's the end of ...      392.0   
392  Right , okay , we'll that's that's the end of ...      393